In [48]:
pip install imblearn

  Using cached https://files.pythonhosted.org/packages/e6/62/08c14224a7e242df2cef7b312d2ef821c3931ec9b015ff93bb52ec8a10a3/imbalanced_learn-0.5.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [5]:
train_feature_files = ["train_RRI_features.csv", "train_templ_features.csv","train_P_hr_features.csv"]
test_feature_files = ["test_RRI_features.csv", "test_templ_features.csv","test_P_hr_features.csv"]

In [6]:
def load_data():
    print("Load the data.")
    global y_train
    global x_train
    global x_test
    
    df_x_train = pd.concat(map(pd.read_csv, train_feature_files),axis=1)
    df_x_test = pd.concat(map(pd.read_csv, test_feature_files),axis=1)
    df_y_train = pd.read_csv("y_train.csv")

    x_test = df_x_test.values[:,1:]
    x_train = df_x_train.values[:,1:]
    y_train = df_y_train['y'].values
    
    print('Standardize the data.')
    scaler = preprocessing.StandardScaler().fit(x_train)
    
    x_train =scaler.transform(x_train)
    
    x_test = scaler.transform(x_test)

In [11]:
# pd.read_csv("test_RRI_features.csv")
load_data()
x_test
print('There are %d NAN values.'%np.sum(np.isnan(x_test)))

Load the data.
Standardize the data.
There are 0 NAN values.


In [12]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
# F1 = f1_score(y_true, y_pred, average='micro')

n=20

# Cvalues = [3,4,5,6,7,8,9,10]
# Cvalues = [5,5.2,5.4,5.6,5.8,6]

def train(C):
    val_result = []

    for i in range(n):
        load_data()
        np.random.seed(i)
        
#         model_RandomUnderSampler = RandomUnderSampler()
#         x_train_t, y_train_t =model_RandomUnderSampler.fit_sample(x_train,y_train)
#         print('After down-sampling:\n',pd.DataFrame(y_train_t,columns=['y']).groupby('y').size())
        
        x_train_t = x_train
        y_train_t = y_train

        x_train_t, x_val_t, y_train_t, y_val_t = train_test_split(x_train_t, y_train_t, test_size=0.1)

        clf = SVC(kernel='rbf',C=C, 
        decision_function_shape='ovr',  gamma='auto',
        max_iter=-1, probability=False, shrinking=True,
        tol=0.001, verbose=False,random_state=i,class_weight=None)   # add class weight

        clf.fit(x_train_t, y_train_t)

        y_val_p = clf.predict(x_val_t)
        val_result.append(f1_score(y_val_t, y_val_p, average='micro'))

        y_pred = clf.predict(x_test)
        print("predicted y:",y_pred)

        f = open("submission_svc{0:.1f}.csv".format(i), "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,x))
        f.close()

    print("C value: %f\nCross validation score:%f"%(C,np.mean(val_result)))

# for C in Cvalues:
#     train(C)
train(5.6)

Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y: [0 0 0 ... 0 0 1]
Load the data.
Standardize the data.
predicted y

In [13]:
size = x_test.shape
result = [list() for i in range(size[0])]


for i in range(n):
    f = "submission_svc%d.0.csv" %(i)
    f = open(f)
    lines = f.readlines()
    for l in lines[1:]:
        l = l.strip().split(',')
        idx, val = int(l[0]), int(float(l[1]))
        result[idx].append(val)
    f.close()
print(result[:20])


def vote(x):
    c = [0] * 4
    for i in x:
        c[i] += 1
    c = [(0, c[0]), (1, c[1]), (2, c[2])]
    c.sort(key = lambda x: x[1], reverse = True)
    if c[0][1] > c[1][1]:
        return c[0][0]
    else:
        if c[0][0] == 1 or c[1][0] == 1:
            return 1
        else:
            return np.random.choice([c[0][0], c[1][0]])
print(vote([1,2,0,0,0]))


with open("voted_svc.csv", "w") as f:
    f.write("id,y\n")
    for i in range(size[0]):
        f.write("{},{}\n".format(i, vote(result[i])))
    f.close()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 